In [ ]:

select * from salary;
select * from income;
select * from deduction;
select * from emp_transaction;

In [ ]:
select s.emp_id, s.emp_name, x.trns_type
, case when x.trns_type = 'Basic' then round(base_salary * (cast(x.percentage as decimal)/100),2)
	   when x.trns_type = 'Allowance' then round(base_salary * (cast(x.percentage as decimal)/100),2)
	   when x.trns_type = 'Others' then round(base_salary * (cast(x.percentage as decimal)/100),2)
	   when x.trns_type = 'Insurance' then round(base_salary * (cast(x.percentage as decimal)/100),2)
	   when x.trns_type = 'Health' then round(base_salary * (cast(x.percentage as decimal)/100),2)
	   when x.trns_type = 'House' then round(base_salary * (cast(x.percentage as decimal)/100),2) end as amount	   
from salary s
cross join (select income as trns_type, percentage from income
			union
			select deduction as trns_type, percentage from deduction) x;

In [ ]:
--building  salary report
with cte as	
	(
		select id, income as trans_type, percentage 
		from
		income
		union all 
		select id, deduction as trans_type, percentage 
		from deduction
	),
	cte2 as
	(
		select emp_id, emp_name, trans_type, (percentage * base_salary)/100 as amount
		from salary
		cross join 
		cte
	),
	cte3 as
	(
select emp_name as employee, 
sum(case when trans_type = 'Basic' then amount else null end) as Basic,
sum(case when trans_type = 'Allowance' then amount else null end) as Allowance,
sum(case when trans_type = 'Others' then amount else null end) as Others,
sum(case when trans_type = 'Insurance' then amount else null end) as Insurance,
sum(case when trans_type = 'Health' then amount else null end) as Health,
sum(case when trans_type = 'House' then amount else null end) as House
from cte2
group by emp_name
)
select *,
Basic + Allowance + Others as Gross,
Insurance + Health + House  as Total_deduction
from cte3